In [4]:
"""General methyl data import using MSM package. General dictonary data, beta value methylation array df, survival metadata df"""

import sys
sys.path.insert(0, '/Users/zacksiegfried/Documents/methylspan')
import MSM

all_files = MSM.pullMethylMetaData('pancreas')

# not gaureteed to return same number of cases, (remove file_id from sample data if missing from meta data)
# missing values dropped anyways in next steps
sample_data = MSM.methylDataFormat(all_files, 30)
meta_data = MSM.metaDataFormat(sample_data, all_files)



Number of methylation files for all cases for primary site pancreas: 430
Final number of cases in sample dataset : 29


In [5]:
### FORMATTING FOR SURVIVAL + PCA

import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# drops all CpG sites with at least 1 missing value
sample_data.index.name = 'id'
sample_data.dropna(axis=1, inplace=True)

# standardizes dataframe
scaled_df = sample_data.copy()
scaled_df = pd.DataFrame(StandardScaler().fit_transform(scaled_df), index=scaled_df.index, columns=scaled_df.columns)

# converts CpG sites to PCA components
pca = PCA(n_components=20)
pca.fit(scaled_df)

output = pca.transform(scaled_df)
x = pd.DataFrame(output, index=scaled_df.index)
x.drop(columns=[14,15,16,17,18,19], inplace=True) # selecting first 14 PCAs

meta_data.index.name = 'id'

# merges dfs and formats 
full_data = pd.merge(meta_data, x, on = 'id')
full_data.replace(['Dead', 'Alive'],[0, 1], inplace=True)
full_data = full_data[full_data['days_to_death'].notna()]

print(full_data.shape)

(27, 16)


In [6]:
### SURVIVAL ANALYSIS EXECUTION & TESTING

from lifelines import CoxPHFitter

cph = CoxPHFitter()
cph.fit(full_data, duration_col='days_to_death', event_col='vital_status')

print(cph.concordance_index_)

0.8983050847457628


/Users/zacksiegfried/opt/anaconda3/lib/python3.9/site-packages/lifelines/fitters/coxph_fitter.py:1611: ConvergenceWarning: Newton-Rhaphson failed to converge sufficiently. Please see the following tips in the lifelines documentation: https://lifelines.readthedocs.io/en/latest/Examples.html#problems-with-convergence-in-the-cox-proportional-hazard-model
  warnings.warn(
/Users/zacksiegfried/opt/anaconda3/lib/python3.9/site-packages/lifelines/fitters/coxph_fitter.py:1266: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  self.params_ = pd.Series(params_, index=pd.Index(X.columns, name="covariate"), name="coef")
